<a href="https://colab.research.google.com/github/cpvivek/Facial-Emotion-Recognition/blob/main/Capstone_DL%26MLE_Facial_Emotion_Recognition_Vivek_CP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Facial Emotion Detection
In this project, we are aiming to develop a CNN netwrok to detect 7 different human emotions in real time over a live web-cam feed. We'll also be deploying a web app to run this model.

Importing necessary libraries including tensorflow, keras, sklearn and opencv

In [1]:
# import necessary files
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
import keras
import tensorflow as tf
from keras.utils import np_utils
from keras.models import Sequential 
from keras.layers import Dense, Conv2D, MaxPooling2D, BatchNormalization, Dropout, Flatten

Importing dataset sourced from kaggle

In [3]:
# mounting drive
from google.colab import drive
drive.mount('/content/drive')

MessageError: ignored

reading the csv file.

In [23]:
df = pd.read_csv('/content/drive/MyDrive/Alma Better Pro/Alma Better Pro Program/Module 4: Machine Learning/Data Sets/icml_face_data.csv')
# for i in range(len(df[' pixels'])):
#   df[' pixels'][i] = np.fromstring(df[' pixels'][i],sep=' ',dtype='float32')
#   df[' pixels'][i] = np.asarray(df[' pixels'][i]).reshape(48,48,1)
# train_data = df[df[' Usage']=='Training']
# val_data = df[df[' Usage']=='PublicTest']
# test_data = df[df[' Usage']=='PrivateTest']

Let's look into our dataset

In [ ]:
df.columns

Index(['emotion', ' Usage', ' pixels'], dtype='object')

So the dataset is not all that complicated.

We've got 'emotion' field indicating different emotions.

I believe since this is a dataset used in kaggle competitions, they've went ahead and done the training, validation and test segregation, labeled by 'Usage' column

And finally in the 'pixels' field, we have the pixelated form of the image, flattened into an 1 dimension array. 

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35887 entries, 0 to 35886
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   emotion  35887 non-null  int64 
 1    Usage   35887 non-null  object
 2    pixels  35887 non-null  object
dtypes: int64(1), object(2)
memory usage: 841.2+ KB


The pixel field is stored as a string at the moment. We'll have to convert that into a 48x48x1 array with float values.

In [24]:
#using lambda function to achieve this conversion
df[' pixels']=df[' pixels'].apply(lambda x: np.fromstring(x, sep=' ',dtype='float32')) #converting string to float separated by ' ' 
df[' pixels']=df[' pixels'].apply(lambda x:np.asarray(x.reshape(48,48,1))) #reshaping to 48x48x1

Let's look into emotions field.

In [25]:
df['emotion'].unique()

array([0, 2, 4, 6, 3, 5, 1])

The dataset contains 7 different emotions labeled using number from 0 to 6.

Let's segregate our training, validation and test sets right away. 

Public Test would be put into validation and Private test into test set. 

In [26]:
df[' Usage'].value_counts()

Training       28709
PublicTest      3589
PrivateTest     3589
Name:  Usage, dtype: int64

In [27]:
#defininng train, val and test data.
training_data=df[df[' Usage']=='Training']
validation_data=df[df[' Usage']=='PublicTest']
testing_data=df[df[' Usage']=='PrivateTest']

In [8]:
training_data['emotion'].value_counts()

3    7215
6    4965
4    4830
2    4097
0    3995
5    3171
1     436
Name: emotion, dtype: int64

We can see that there is considerable amount of imbalance in the dataset wrt to emotions in training dataset. This will create bias in the model and result in misclassifying. 

We can treat this issue by data augmentation.

Using keras for data augmentation.

x_train formation


In [9]:
x_train=[]
for i in range(len(training_data[' pixels'])):
  x_train.append(training_data[' pixels'][i])
x_train=np.asarray(x_train) #since we need arrays as input to CNN 
x_train=x_train.reshape(len(x_train),48,48,1)
y_train=np.array(training_data['emotion'])
y_train=y_train.astype(int)
y_train=np_utils.to_categorical(y_train,7)
# shape of training data
x_train.shape, y_train.shape

((28709, 48, 48, 1), (28709, 7))

x_val

In [28]:
x_val = []
for i in validation_data[' pixels']:
  x_val.append(i)
x_val=np.asarray(x_val)
x_val= x_val.reshape(len(x_val),48,48,1)
y_val=np.array(validation_data['emotion'])
y_val=y_val.astype(int)
y_val=np_utils.to_categorical(y_val,7)



x_val.shape,y_val.shape

((3589, 48, 48, 1), (3589, 7))

Preprocessing x_test using keras

In [29]:
x_test = []

for i in testing_data[' pixels']:
  x_test.append(i)
x_test=np.array(x_test)
x_test=x_test.reshape(len(x_test),48,48,1)
y_test=np.array(testing_data['emotion'])
y_test=y_test.astype(int)
y_test=np_utils.to_categorical(y_test,7)
# code here

# then check shape
x_test.shape,y_test.shape

((3589, 48, 48, 1), (3589, 7))

Data Augmentation Process.

We can observe that there is a imbalance in labels in train, val and test sets. This will result in bias in the model twoards certain emotions.

We'll be able to sort this issue adding augmented images to data to deal with imbalance.

In [31]:
from keras.preprocessing.image import ImageDataGenerator

datagen= ImageDataGenerator(
    rescale=1./255, #rescaling
    rotation_range=10,#rotating the image
    horizontal_flip=True,#flipping the image horizontally
    width_shift_range=0.1,
    height_shift_range=0.1, #shift in width and height
    fill_mode='nearest'
)
testgen= ImageDataGenerator(rescale=1./255)

In [13]:
datagen.fit(x_train)

In [14]:
train_flow=datagen.flow(x_train,y_train,batch_size=64)
test_flow=testgen.flow(x_test,y_test,batch_size=64)

Building models usign keras layers and keras sequential model

In [32]:
from keras.models import Model
from keras.layers import Input, Dense, Flatten, Dropout, BatchNormalization
from keras.layers.convolutional import Conv2D
from keras.layers.pooling import MaxPooling2D
from keras.layers.merge import concatenate

from keras.regularizers import l1, l2
from matplotlib import pyplot as plt
from sklearn.metrics import confusion_matrix


In [16]:
def FER_Model(input_shape=(48,48,1)):
  visible=Input(shape=input_shape,name='input')
  num_classes=7

  #1st block
  conv1_1=Conv2D(64,kernel_size=3, activation='relu', padding='same', name='conv1_1')(visible)
  conv1_1=BatchNormalization()(conv1_1)
  conv1_2=Conv2D(64,kernel_size=3,activation='relu', padding='same', name='conv1_2')(conv1_1)
  conv1_2=BatchNormalization()(conv1_2)
  pool1_1=MaxPooling2D(pool_size=(2,2), name='pool1_1')(conv1_2)
  drop1_1 = Dropout(0.3, name = 'drop1_1')(pool1_1)#the 2-nd block

#2nd Block 
  conv2_1 = Conv2D(128, kernel_size=3, activation='relu', padding='same', name = 'conv2_1')(drop1_1)
  conv2_1 = BatchNormalization()(conv2_1)
  conv2_2 = Conv2D(128, kernel_size=3, activation='relu', padding='same', name = 'conv2_2')(conv2_1)
  conv2_2 = BatchNormalization()(conv2_2)
  conv2_3 = Conv2D(128, kernel_size=3, activation='relu', padding='same', name = 'conv2_3')(conv2_2)
  conv2_2 = BatchNormalization()(conv2_3)
  pool2_1 = MaxPooling2D(pool_size=(2,2), name = 'pool2_1')(conv2_3)
  drop2_1 = Dropout(0.3, name = 'drop2_1')(pool2_1)#the 3-rd block
  
  #3rd block

  conv3_1 = Conv2D(256, kernel_size=3, activation='relu', padding='same', name = 'conv3_1')(drop2_1)
  conv3_1 = BatchNormalization()(conv3_1)
  conv3_2 = Conv2D(256, kernel_size=3, activation='relu', padding='same', name = 'conv3_2')(conv3_1)
  conv3_2 = BatchNormalization()(conv3_2)
  conv3_3 = Conv2D(256, kernel_size=3, activation='relu', padding='same', name = 'conv3_3')(conv3_2)
  conv3_3 = BatchNormalization()(conv3_3)
  conv3_4 = Conv2D(256, kernel_size=3, activation='relu', padding='same', name = 'conv3_4')(conv3_3)
  conv3_4 = BatchNormalization()(conv3_4)
  pool3_1 = MaxPooling2D(pool_size=(2,2), name = 'pool3_1')(conv3_4)
  drop3_1 = Dropout(0.3, name = 'drop3_1')(pool3_1)#the 4-th block
 
 #4th block
 
  conv4_1 = Conv2D(256, kernel_size=3, activation='relu', padding='same', name = 'conv4_1')(drop3_1)
  conv4_1 = BatchNormalization()(conv4_1)
  conv4_2 = Conv2D(256, kernel_size=3, activation='relu', padding='same', name = 'conv4_2')(conv4_1)
  conv4_2 = BatchNormalization()(conv4_2)
  conv4_3 = Conv2D(256, kernel_size=3, activation='relu', padding='same', name = 'conv4_3')(conv4_2)
  conv4_3 = BatchNormalization()(conv4_3)
  conv4_4 = Conv2D(256, kernel_size=3, activation='relu', padding='same', name = 'conv4_4')(conv4_3)
  conv4_4 = BatchNormalization()(conv4_4)
  pool4_1 = MaxPooling2D(pool_size=(2,2), name = 'pool4_1')(conv4_4)
  drop4_1 = Dropout(0.3, name = 'drop4_1')(pool4_1)
    
  #the 5-th block
  
  conv5_1 = Conv2D(512, kernel_size=3, activation='relu', padding='same', name = 'conv5_1')(drop4_1)
  conv5_1 = BatchNormalization()(conv5_1)
  conv5_2 = Conv2D(512, kernel_size=3, activation='relu', padding='same', name = 'conv5_2')(conv5_1)
  conv5_2 = BatchNormalization()(conv5_2)
  conv5_3 = Conv2D(512, kernel_size=3, activation='relu', padding='same', name = 'conv5_3')(conv5_2)
  conv5_3 = BatchNormalization()(conv5_3)
  conv5_4 = Conv2D(512, kernel_size=3, activation='relu', padding='same', name = 'conv5_4')(conv5_3)
  conv5_3 = BatchNormalization()(conv5_3)
  pool5_1 = MaxPooling2D(pool_size=(2,2), name = 'pool5_1')(conv5_4)
  drop5_1 = Dropout(0.3, name = 'drop5_1')(pool5_1)#Flatten and output
  flatten = Flatten(name = 'flatten')(drop5_1)
  ouput = Dense(num_classes, activation='softmax', name = 'output')(flatten)# create model 
  model = Model(inputs =visible, outputs = ouput)
   # summary layers
  print(model.summary())
    
  return model

In [17]:
model=FER_Model()
opt=tf.keras.optimizers.Adam(lr=0.0001,decay=1e-6)
model.compile(loss='categorical_crossentropy',optimizer=opt, metrics=['accuracy'])

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input (InputLayer)          [(None, 48, 48, 1)]       0         
                                                                 
 conv1_1 (Conv2D)            (None, 48, 48, 64)        640       
                                                                 
 batch_normalization (BatchN  (None, 48, 48, 64)       256       
 ormalization)                                                   
                                                                 
 conv1_2 (Conv2D)            (None, 48, 48, 64)        36928     
                                                                 
 batch_normalization_1 (Batc  (None, 48, 48, 64)       256       
 hNormalization)                                                 
                                                                 
 pool1_1 (MaxPooling2D)      (None, 24, 24, 64)        0     

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [ ]:
num_epochs = 100  
history = model.fit_generator(train_flow, 
                    steps_per_epoch=len(x_train) / 64, 
                    epochs=num_epochs,  
                    verbose=1,  
                    validation_data=test_flow)


In [19]:
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
model.save_weights("model.h5")
print("Saved model to disk")

Saved model to disk


Transfer Learning

In [35]:
model=tf.keras.applications.MobileNetV2()

14548992/14536120 [==============================] - 0s 0us/step


In [ ]:
img_size=224
new_array=cv2.resize(_ )